# In this notebook I was trying out different parameter values for training the model

In [1]:
import detectron2
import torch, torchvision
from utils.dataset import *
from utils.evaluator import *
from utils.trainer import MTSDTrainer

from detectron2.data import DatasetCatalog
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
import matplotlib.pyplot as plt

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.engine import DefaultTrainer
from datetime import datetime


from detectron2_backbone import backbone
from detectron2_backbone.config import add_backbone_config

#import the COCO Evaluator to use the COCO Metrics
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

In [2]:
#l_id_f="/home/pkost/workspace/inzynierka/inz/lbl_id_dict.pkl"
l_id_f="/home/pkos/workspace/inz/praca_inzynierska/lbl_id_dict.pkl"
with open(l_id_f, 'rb') as f:
        lbl_id_map = pickle.load(f)
labels = list(lbl_id_map.keys())

In [3]:
with open('dataset_dicts.pkl', 'rb') as f:
    dataset_dicts = pickle.load(f)

In [4]:
ann_d="/mnt/CommonData/dataset/filtered_annotations_2/"
#img_d="/mnt/CommonData/dataset/images/"
#img_d="/mnt/CommonData/dataset/mtsd_fully_annotated/images/"
img_d="/home/pkos/Desktop/mtsd_fully_annotated/images"
for d in ["train", "val"]:
    #id_f="/mnt/CommonData/dataset/splits/"+ d +".txt"
    #id_f="/mnt/CommonData/dataset/mtsd_fully_annotated/splits/"+ d +".txt"
    id_f="/home/pkos/Desktop/mtsd_fully_annotated/splits/" + d +".txt"
    
    if d=="train":
        DatasetCatalog.register("MTSD_"+d, 
                            lambda id_f=id_f, ann_d=ann_d, img_d=img_d, l_id_f=l_id_f: 
                                prepare_MTSD_for_detectron(id_f, ann_d, img_d, l_id_f))
    else:
        DatasetCatalog.register("MTSD_"+d, 
                            lambda id_f=id_f, ann_d=ann_d, img_d=img_d, l_id_f=l_id_f: 
                                prepare_MTSD_for_detectron(id_f, ann_d, img_d, l_id_f))
    MetadataCatalog.get("MTSD_" + d).set(thing_classes=labels)
    
signs_metadata = MetadataCatalog.get("MTSD_train")
#later, to access the data:
#data: List[Dict] = DatasetCatalog.get("my_dataset")

In [5]:
#TODO:poiteruj po roznych kombinacjach wartosci parametrów i ucz na podzbiorze treningowego przez 40k iteracji i wybierz najlepsza kombinacje
for (ims_pb,lr,res) in  [(2,0.0015, (704, 720, 774, 809, 844, 880))]: #(2,0.001, (704, 720, 774, 809, 844, 880)),#(3,0.001, (480, 504, 528, 552, 576, 600))]:
    #na poczatku dla ipb=2 bylo: (640, 672, 704, 736, 768, 800)
    print(ims_pb, lr)
    cfg = get_cfg()
    #add_backbone_config(cfg)

    cfg.OUTPUT_DIR =f"./tboard_logs/param_tuning_{ims_pb}_{lr}_2/"
    #cfg.OUTPUT_DIR = f"./tboard_logs/retinanet_augm_dev/"
    #to potrzebuje 4gb:
    cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/retinanet_R_50_FPN_3x.yaml"))

    #Fast_rcnn odpada - https://github.com/facebookresearch/detectron2/issues/1279
    #cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/fast_rcnn_R_50_FPN_1x.yaml"))

    cfg.DATASETS.TRAIN = ("MTSD_train",)
    cfg.DATASETS.TEST = ("MTSD_val",)
    cfg.DATALOADER.NUM_WORKERS = 2

    # cfg.MODEL.BACKBONE.NAME = "build_fcos_resnet18_fpn_backbone"
    #cfg.MODEL.BACKBONE.NAME = "build_fcos_mnv2_fpn_backbone"

    # cfg.MODEL.BACKBONE.NAME = "build_fcos_efficientnet_fpn_backbone"
    #cfg.MODEL.DEVICE="cuda"

    cfg.MODEL.WEIGHTS= model_zoo.get_checkpoint_url("COCO-Detection/retinanet_R_50_FPN_3x.yaml")  #None # Let training initialize from model zoo
    cfg.MODEL.META_ARCHITECTURE = 'RetinaNet'
    cfg.MODEL.RETINANET.NUM_CLASSES = len(labels) + 1
    ####MODEL######################
    #---------------------------#
    #TODO:policz srednie i std--#
    #---------------------------#
    #############################

    cfg.SOLVER.IMS_PER_BATCH = ims_pb #default 16
    cfg.SOLVER.BASE_LR = lr  # pick a good LR
    cfg.SOLVER.WARMUP_ITERS = 4000
    cfg.SOLVER.GAMMA = 0.3
    cfg.SOLVER.STEPS = (12_000, 40_000, 60_000, 80_000)
    cfg.SOLVER.MAX_ITER = 50_000 #300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
    #cfg.MODEL.RETINANET.IMS_PER_BATCH = 2
    #cfg.MODEL.RETINANET.BATCH_SIZE_PER_IMAGE = 64   #faster, and good enough for this toy dataset (default: 512)
    # cfg.INPUT.CROP.ENABLED=True
    # cfg.INPUT.CROP.SIZE=[0.7, 0.7]


    cfg.INPUT.MIN_SIZE_TRAIN = res#
    cfg.INPUT.MIN_SIZE_TEST = 648#(256,) #def 800
    cfg.INPUT.MAX_SIZE_TRAIN = 1920#(1024,) #def 1333
    cfg.INPUT.MAX_SIZE_TEST = 1920#(1024,) #def 1333

    cfg.TEST.EVAL_PERIOD = 5000 #200
    cfg.SOLVER.CHECKPOINT_PERIOD = 5_000
    cfg.VIS_PERIOD = 5_000
    cfg.TEST.DETECTIONS_PER_IMAGE = 10
    os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
    with open(cfg.OUTPUT_DIR + "/config.yml", "w") as f:
        f.write(cfg.dump())
    torch.cuda.empty_cache()
    trainer = MTSDTrainer(cfg) 
    trainer.resume_or_load(resume=True)#(resume=False)
    trainer.train()

Loading config /home/pkos/miniconda3/envs/roadeye/lib/python3.8/site-packages/detectron2/model_zoo/configs/COCO-Detection/../Base-RetinaNet.yaml with yaml.unsafe_load. Your machine may be at risk if the file contains malicious content.


2 0.0015
[09/23 18:50:48 d2.engine.defaults]: Model:
RetinaNet(
  (backbone): FPN(
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelP6P7(
      (p6): Conv2d(2048, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (p7): Conv2d(256, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    )
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
   

/home/pkos/miniconda3/envs/roadeye/lib/python3.8/site-packages/detectron2/layers/wrappers.py:226: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /opt/conda/conda-bld/pytorch_1595629411241/work/torch/csrc/utils/python_arg_parser.cpp:766.)
  return x.nonzero().unbind(1)


[09/23 18:51:48 d2.utils.events]:  eta: 3:50:46  iter: 25019  total_loss: 0.329  loss_cls: 0.232  loss_box_reg: 0.123  time: 0.5574  data_time: 0.0346  lr: 0.000450  max_mem: 5783M
[09/23 18:52:00 d2.utils.events]:  eta: 3:54:06  iter: 25039  total_loss: 0.428  loss_cls: 0.298  loss_box_reg: 0.140  time: 0.5642  data_time: 0.0515  lr: 0.000450  max_mem: 5784M
[09/23 18:52:11 d2.utils.events]:  eta: 3:53:54  iter: 25059  total_loss: 0.463  loss_cls: 0.301  loss_box_reg: 0.158  time: 0.5581  data_time: 0.0043  lr: 0.000450  max_mem: 5790M
[09/23 18:52:22 d2.utils.events]:  eta: 3:49:37  iter: 25079  total_loss: 0.589  loss_cls: 0.391  loss_box_reg: 0.201  time: 0.5527  data_time: 0.0118  lr: 0.000450  max_mem: 6982M
[09/23 18:52:33 d2.utils.events]:  eta: 3:44:17  iter: 25099  total_loss: 0.466  loss_cls: 0.312  loss_box_reg: 0.185  time: 0.5467  data_time: 0.0041  lr: 0.000450  max_mem: 6982M
[09/23 18:52:43 d2.utils.events]:  eta: 3:42:26  iter: 25119  total_loss: 0.405  loss_cls: 0.25

In [ ]:
#MTSDevaluator cos tam liczy, ale nw czemu tensorboard tego nie pokazuje

In [15]:
%matplotlib qt
from detectron2.utils.visualizer import ColorMode
import time
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth") 
cfg.MODEL.RETINANET.SCORE_THRESH_TEST = 0.4   # set a custom testing threshold
predictor = DefaultPredictor(cfg)
dataset_dicts_test = prepare_MTSD_for_detectron("/mnt/CommonData/dataset/mtsd_fully_annotated/splits/val.txt", ann_d, img_d, l_id_f)

In [34]:
# for d in random.sample(dataset_dicts, 1):

#dataset_dicts_test = list(filter(lambda d: any(annot["category_id"] in (66,-1) for annot in d["annotations"]), dataset_dicts_test))
#print(len(dataset_dicts_test))
i=0
while True:
    i=i+1
    d = random.sample(dataset_dicts_test, 1)[0]
    #print(predictor.model([mapper(d)]))
    im = cv2.imread(d["file_name"])
    #im = cv2.imread("/home/pkos/Desktop/VOCABULARY-TRafficSigns.png")

    print("l. znakow:",len(d['annotations']))
    millis = int(round(time.time() * 1000))
    outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
    #with torch.no_grad():
    #    outputs=predictor.model([mapper(d)])[0]
    print(int(round(time.time() * 1000)) - millis)
    #print(outputs)
#     if len(outputs["instances"]) == 0:
#         continue
    
    v = Visualizer(im[:, :, ::-1],               
        metadata=signs_metadata, 
        scale=0.8   # remove the colors of unsegmented pixels. This option is only available for segmentation models
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    plt.figure(figsize=(10,8))
    plt.imshow(out.get_image())
    #     v.draw_
    print()
    print(outputs)
    break


l. znakow: 4
172

{'instances': Instances(num_instances=2, image_height=3936, image_width=5248, fields=[pred_boxes: Boxes(tensor([[3568.5779, 2337.0964, 3708.8208, 2472.6965],
        [3303.3433, 2533.0466, 3334.4744, 2564.3823]], device='cuda:0')), scores: tensor([0.8216, 0.4073], device='cuda:0'), pred_classes: tensor([61, 61], device='cuda:0')])}
